In [16]:
# Install necessary packages if not already installed:
# pip install transformers datasets sklearn

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# 1. Load the sentiment/emotion dataset
dataset = load_dataset("emotion")  # 6 emotions: sadness, joy, love, anger, fear, surprise
# If using GoEmotions, you could do: dataset = load_dataset("go_emotions", "simplified") and map 27 emotions to 4 sentiments here.

# 2. Pre-process: Tokenize the text
checkpoint = "distilbert-base-uncased"  # Using DistilBERT for efficiency
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_batch(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)

# Tokenize the dataset (this adds 'input_ids' and 'attention_mask' columns)
tokenized_dataset = dataset.map(tokenize_batch, batched=True)

# 3. Set format for PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 4. Load the pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)

# 5. Define training hyperparameters and Trainer
batch_size = 16
training_args = TrainingArguments(
    output_dir="sentiment_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_steps=50,
    log_level="error"  # only show errors to avoid too much log output
)

# Metric function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    return {"accuracy": acc, "macro_f1": macro_f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

# 6. Train the model
trainer.train()

# 7. Evaluate on the test set
results = trainer.evaluate(tokenized_dataset["test"])
print(f"Test accuracy: {results['eval_accuracy']:.4f}")
print(f"Test Macro-F1: {results['eval_macro_f1']:.4f}")


c:\Second_Sem\csc392\env_py310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saadu\.cache\huggingface\hub\datasets--emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 795505.74 examples/s]
c:\Second_Sem\csc392\env_py310\lib\site-pac

{'loss': 1.5308, 'learning_rate': 1.9750000000000002e-05, 'epoch': 0.05}


  3%|▎         | 102/4000 [00:03<01:55, 33.82it/s]

{'loss': 1.2819, 'learning_rate': 1.951e-05, 'epoch': 0.1}


  4%|▍         | 154/4000 [00:04<01:49, 35.17it/s]

...

{'eval_loss': 0.18607589602470398, 'eval_accuracy': 0.937, 'eval_macro_f1': 0.9125431369879897, 'eval_runtime': 0.7278, 'eval_samples_per_second': 2748.124, 'eval_steps_per_second': 171.758, 'epoch': 4.0}


100%|██████████| 4000/4000 [02:00<00:00, 33.11it/s]


{'train_runtime': 120.8143, 'train_samples_per_second': 529.739, 'train_steps_per_second': 33.109, 'train_loss': 0.2095928320288658, 'epoch': 4.0}


100%|██████████| 125/125 [00:00<00:00, 164.23it/s]

Test accuracy: 0.9235
Test Macro-F1: 0.8740


In [19]:
trainer.save_model("final_model_directory")
tokenizer.save_pretrained("final_model_directory")


('final_model_directory\\tokenizer_config.json',
 'final_model_directory\\special_tokens_map.json',
 'final_model_directory\\vocab.txt',
 'final_model_directory\\added_tokens.json',
 'final_model_directory\\tokenizer.json')

In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Load the saved model and tokenizer
model_dir = "final_model_directory"  # or use the output_dir you used during training
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Function to predict sentiment for an individual prompt
def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    
    # Get model predictions (logits)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    # Convert logits to probabilities using softmax
    probs = torch.nn.functional.softmax(logits, dim=-1)
    
    # Get the predicted label index
    pred_label_idx = int(np.argmax(probs.numpy(), axis=-1)[0])
    
    # Map label indices to label names (assume same ordering as in training)
    label_names = ["sadness", "joy", "love", "anger", "fear", "surprise"]  # update if needed
    pred_label = label_names[pred_label_idx]
    return pred_label, probs.numpy()[0]

# Test on individual prompts
text_examples = [
    "I feel so happy and excited about life!",
    "I'm really upset and angry with what happened.",
    "I love spending time with my family.",
    "I am terrified of the dark.",
    "The news left me completely shocked.",
    "I'm feeling a bit down today.",
    "I want a burger."
]

for text in text_examples:
    label, prob = predict_sentiment(text)
    print(f"Text: {text}\nPredicted Sentiment: {label}\nProbabilities: {prob}\n")


Text: I feel so happy and excited about life!
Predicted Sentiment: joy
Probabilities: [6.6874069e-05 9.9965417e-01 1.0660033e-04 5.1774790e-05 2.8712684e-05
 9.1870250e-05]

Text: I'm really upset and angry with what happened.
Predicted Sentiment: anger
Probabilities: [5.12196450e-03 3.49217822e-04 1.15422496e-04 9.91796374e-01
 2.46886816e-03 1.48101841e-04]

Text: I love spending time with my family.
Predicted Sentiment: joy
Probabilities: [0.05378823 0.5414191  0.07344555 0.30905557 0.01956067 0.00273079]

Text: I am terrified of the dark.
Predicted Sentiment: fear
Probabilities: [1.7986086e-04 1.3891693e-04 7.8151359e-05 4.0574523e-04 9.9888700e-01
 3.1029904e-04]

Text: The news left me completely shocked.
Predicted Sentiment: surprise
Probabilities: [4.7630229e-04 1.1978641e-03 3.7912233e-04 1.3507752e-03 2.1683846e-03
 9.9442756e-01]

Text: I'm feeling a bit down today.
Predicted Sentiment: sadness
Probabilities: [9.9840039e-01 4.4780798e-04 9.9361489e-05 5.6936336e-04 3.8814213